<a href="https://colab.research.google.com/github/kjgeorge88/colab-cfd-tests/blob/master/diffusion_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import time

# simulation parameters
scale  = 2                 # set simulation size
NX     = 32*scale          # domain size 
NY     = NX                # domain size
dx     = 1                 # Grid spacing
dt     = 0.25              # Time step  
NSTEPS = 2000*scale*scale  # number of simulation time steps
NMSG   = 500*scale*scale   # show messages every NMSG time steps
vis    = True              # show visualisation; set to False for performance measurements
NVIS   = NMSG              # show visualisation every NVIS time steps
u_max  = 0.04/scale        # maximum velocity
nu     = 1                 # kinematic shear viscosity

In [0]:
# Mesh parameters
x = np.arange(NX)+0.5
y = np.arange(NY)+0.5
[X, Y] = np.meshgrid(x,y)

In [0]:
# Taylor Green Vortex 
def taylorgreen(t, nu, u_max):
    kx = 2*np.pi/NX
    ky = 2*np.pi/NY
    td = 1/(nu*(kx*kx+ky*ky))
    u = -u_max*np.sqrt(ky/kx)*np.cos(kx*X)*np.sin(ky*Y)*np.exp(-t/td)   
    return u

In [0]:
# Initial Conditions
u = taylorgreen(0, nu, u_max)

print('Solving Diffusion Equation')
print('      domain size:', str(NX) + 'x' + str(NY))
print('               nu:', nu)
print('              tau:', tau)
print('            u_max:', u_max)
print('             rho0:', rho0)
print('        timesteps:', NSTEPS)
print('       plot every:', NVIS)
print('    message every:', NMSG)
print('')

tstart = time.time()

In [0]:
import numba

@numba.stencil
def _diffusion(x,nu,dt,dx):
    dtbydx2 = dt/(dx*dx)
    return (1-4.0*dtbydx2*nu)*x[0,0] + dtbydx2*nu*(            x[0, 1] +
                                                    x[-1, 0] +            x[1, 0] +
                                                               x[0,-1]             ) 

@numba.njit
def diffusion_cpu(x,nu,dt,dx):
    return _diffusion(x,nu,dt,dx)

In [0]:
#x_cpu = np.ones((64, 64), dtype='int8')

#%timeit diffusion_cpu(x_cpu,nu,dt,dx)

In [0]:
# Main loop
for i in range(1, NSTEPS+1):
    
    # Advance in time
    t = i*dt
    #u = taylorgreen(t, nu, u_max)
    u = diffusion_cpu(u,nu,dt,dx)
     
    # Compare against the analytical solution every NMSG time steps
    if i % NMSG == 0:
        
        # Calculate analytical solution
        ua = taylorgreen(t, nu, u_max)

        sumue2 = np.einsum('xy->', (u-ua)*(u-ua))        
        L2e    = np.sqrt(sumue2)
        
        print(t, L2e)

# Calculate performance information after the simulation is finished
runtime = time.time() - tstart
nodes_updated = NSTEPS*NX*NY
speed = nodes_updated/(1e6*runtime)

print(' ----- performance information -----')
print('        timesteps:', NSTEPS);
print('          runtime:', runtime, '(s)')
print('            speed:', speed, '(Mlups)')

In [0]:
if vis:
    umag = np.sqrt( np.einsum('xy->xy', u*u) ) / u_max
    plt.imshow(umag, extent=[0,1,0,1], vmin=0, vmax=1)
    bar = plt.colorbar()
    plt.xlabel('$x/l_x$')
    plt.ylabel('$y/l_y$')
    bar.set_label('$|\mathbf{u}|/u_\mathrm{max}$')
    td = 1/(nu*(2*np.pi/NX)**2 + (2*np.pi/NY)**2)
    plt.title('Field at $t/t_d = {0:.2f}$'.format(t/td))
    plt.show()